In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.ensemble import IsolationForest
import seaborn as sns
from df_functions import read_csv_file, extract_users, format_data_req, clean_reqlogs

**Data pre-processing**

Examine:
- the time between requests
- number of requests
- divide data into smaller intervals
- check number of tokens within a 5-minute time frame
- number of requests in a row

In [20]:
"""
#This is the "old" code. 
data = pd.read_csv('requests.csv')
data.columns = ['timestamp', 'userID', 'sessionID', 'expiring', 'URL']
data

"""
data = read_csv_file('requests.csv')
#data

users = extract_users(data)
#users

formatted_df = format_data_req(data, 'URL')
#formatted_df

df_cleaned = clean_reqlogs(formatted_df)
df_cleaned

NameError: name 're' is not defined

In [134]:
"""
OLD CODE - step done in box above

#Formatting the data frame so that each row represent one user. 
#All requests are saved as a string in the "URL"-column
users = list(data['userID'])
allUsers = data.set_index(['userID']).rename_axis(None)
allUsers = allUsers.drop(columns=['timestamp', 'sessionID', 'expiring'])
df = allUsers.groupby(level=0).agg(','.join)
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'users'})
"""


In [16]:
"""
OLD CODE - step done in box above

#Removing all non-letter characters from the data and assigning it to new data frame 'df_cleaned'
df_cleaned = df
request_logs = df_cleaned['URL']

cleaned_logs = []

for i in range(0, len(request_logs)):
    sequence = re.sub('[^a-zA-Z]', ' ', request_logs[i])
    sequence = sequence.lower()
    cleaned_logs.append(sequence)

df_cleaned ['request_logs'] = cleaned_logs
df_cleaned = df_cleaned.drop('URL', axis=1)

# 5 and 6: spammers, 7: data scraper
#print(cleaned_logs[7])

"""

NameError: name 'df' is not defined

In [136]:
#Using the tfidf-vectorizer
tfidf_vect = CountVectorizer()
#tfidf_vect = TfidfVectorizer()
vect_logs = tfidf_vect.fit_transform(df_cleaned['request_logs'])

In [137]:
#Code for checking an instance's request_logs

vector_instance=vect_logs[2] 
df_vectorized = pd.DataFrame(vector_instance.T.todense(), index=tfidf_vect.get_feature_names_out(), columns=["tfidf"])
df_vectorized.sort_values(by=["tfidf"], ascending=False)


,tfidf
conversation,34
profile,26
page,20
markasread,17
chat,15
send,15
searchusers,11
inbox,9
login,7
block,4



Some annotations for later
variance normal user[0]: 0.030732589
variance normal user[1]: 0.040072223
variance spammer 1: 0.075907392
variance spammer 2: 0.075314139
variance data scraper: 0.25440979



In [138]:
df_tfidf_scores = pd.DataFrame(vect_logs.todense(), columns= tfidf_vect.get_feature_names_out())
users = df['users']
#df_complete = pd.concat([df, df_tfidf_scores], axis=1)
#df_complete = df_complete.drop('request_logs', axis=1)
#df_complete

#Tror att vi vill hålla dem separat så att vi bara har numeriska värden att köra in i modellen. Men nu finns det en korrekt indexerad lista på alla användare:)
#print(users)
df_tfidf_scores

,block,chat,config,conversation,inbox,like,login,markasread,page,photos,profile,report,searchusers,send,userconfig
0,2,15,1,30,4,0,7,15,18,0,25,3,14,15,1
1,0,13,0,34,2,1,5,17,11,1,23,1,9,13,0
2,4,15,2,34,9,2,7,17,20,2,26,2,11,15,2
3,0,16,2,30,1,1,7,15,9,1,33,4,8,16,2
4,1,9,0,32,1,1,6,16,12,1,25,3,11,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,2,27,6,52,3,6,14,26,26,6,60,3,23,27,6
1000,7,52,4,100,8,4,15,50,39,4,87,2,31,52,4
1001,1,8,1,32,3,2,6,16,13,2,23,0,10,8,1
1002,0,12,0,28,8,2,5,14,16,2,22,0,8,12,0


In [114]:
variance_columns = ['profile', 'config', 'page', 'searchusers', 'block', 'chat', 'conversation', 'inbox', 'report', 'send', 'like', 'login', 'markasread', 'photos', 'userconfig']
df_tfidf_scores['var_reqs'] = df_tfidf_scores[variance_columns].apply(lambda row: row.var(), axis=1)


In [115]:
df_tfidf_scores

,block,chat,config,conversation,inbox,like,login,markasread,page,photos,profile,report,searchusers,send,userconfig,var_reqs
0,2,15,1,30,4,0,7,15,18,0,25,3,14,15,1,92.857143
1,0,13,0,34,2,1,5,17,11,1,23,1,9,13,0,101.380952
2,4,15,2,34,9,2,7,17,20,2,26,2,11,15,2,98.314286
3,0,16,2,30,1,1,7,15,9,1,33,4,8,16,2,110.380952
4,1,9,0,32,1,1,6,16,12,1,25,3,11,9,0,93.266667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,2,27,6,52,3,6,14,26,26,6,60,3,23,27,6,322.409524
1000,7,52,4,100,8,4,15,50,39,4,87,2,31,52,4,1022.828571
1001,1,8,1,32,3,2,6,16,13,2,23,0,10,8,1,85.971429
1002,0,12,0,28,8,2,5,14,16,2,22,0,8,12,0,75.685714


In [116]:
unique_reqs_columns = ['profile', 'config', 'page', 'searchusers', 'block', 'chat', 'conversation', 'inbox', 'report', 'send', 'like', 'login', 'markasread', 'photos', 'userconfig']
unique_reqs = np.count_nonzero(df_tfidf_scores[unique_reqs_columns], axis=1)
df_tfidf_scores['unique_reqs'] = unique_reqs

In [117]:
#Code to join the df_cleaned with the tf-idf scores. Try this with Isolation forest if we need to
joined_df = pd.concat([df_cleaned, df_tfidf_scores], axis = 1)
joined_df

,users,request_logs,block,chat,config,conversation,inbox,like,login,markasread,page,photos,profile,report,searchusers,send,userconfig,var_reqs,unique_reqs
0,user-10,login searchusers page profile c...,2,15,1,30,4,0,7,15,18,0,25,3,14,15,1,92.857143,13
1,user-100,login searchusers page profile p...,0,13,0,34,2,1,5,17,11,1,23,1,9,13,0,101.380952,12
2,user-1000,login searchusers page profile c...,4,15,2,34,9,2,7,17,20,2,26,2,11,15,2,98.314286,15
3,user-1001,login searchusers page profile p...,0,16,2,30,1,1,7,15,9,1,33,4,8,16,2,110.380952,14
4,user-1002,login searchusers page profile c...,1,9,0,32,1,1,6,16,12,1,25,3,11,9,0,93.266667,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,user-995,login searchusers page profile p...,2,27,6,52,3,6,14,26,26,6,60,3,23,27,6,322.409524,15
1000,user-996,login searchusers page profile p...,7,52,4,100,8,4,15,50,39,4,87,2,31,52,4,1022.828571,15
1001,user-997,login searchusers page profile c...,1,8,1,32,3,2,6,16,13,2,23,0,10,8,1,85.971429,14
1002,user-998,login searchusers page profile p...,0,12,0,28,8,2,5,14,16,2,22,0,8,12,0,75.685714,11


**Isolation Forest**

In [70]:
df_anomaly = df_tfidf_scores

#all features as anomaly inputs
#anomaly_inputs = ['profile', 'page', 'searchusers', 'block', 'chat', 'config', 'conversation', 'inbox', 'report', 'send', 'userconfig', 'like', 'login', 'markasread', 'photos', 'unique_reqs', 'var_reqs']

#selected features as anomaly inputs (most significant features fpr user 903 are removed)
#anomaly_inputs = ['profile', 'page', 'searchusers', 'block', 'chat', 'conversation', 'inbox', 'report', 'send', 'like', 'login', 'markasread', 'photos', 'userconfig', 'var_reqs', 'unique_reqs']

#only higher level features as inputs
anomaly_inputs = ['unique_reqs', 'var_reqs']

model_if = IsolationForest(contamination=0.002, random_state=42)
model_if.fit(df_anomaly[anomaly_inputs])

c:\Users\rosin\anaconda3\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


IsolationForest(contamination=0.002, random_state=42)

In [71]:
df_anomaly['anomaly_score'] = model_if.decision_function(df_anomaly[anomaly_inputs])
df_anomaly['anomaly'] = model_if.predict(df_anomaly[anomaly_inputs])

#This dataframe will only include anomaly score and anomaly label
df_anomaly_score = df_anomaly.loc[:,['anomaly_score', 'anomaly']]
df_anomaly_score.loc[df_anomaly_score['anomaly']==-1] 

,anomaly_score,anomaly
5,-0.001777,-1
6,-0.000645,-1
7,-0.019517,-1


In [73]:
#Plots showcasing the outliers detected by isolation forest
palette = ['#ff7f0e', '#1f77b4']
sns.pairplot(df_tfidf_scores, vars=anomaly_inputs, hue='anomaly', palette=palette)

**Testing out TSNE**

In [16]:
#Visualizing data with t-SNE
embedded_data = TSNE(perplexity=50.0, n_iter=2000).fit_transform(np.asarray(vect_logs.todense()))
spammer1 = embedded_data[5]
spammer2 = embedded_data[6]
dataScraper = embedded_data[7]

c:\Users\rosin\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\rosin\anaconda3\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [17]:
plt.scatter(embedded_data[:,0], embedded_data[:,1])
plt.scatter(spammer1[0],spammer1[1], color='red')
plt.scatter(spammer2[0],spammer2[1], color='red')
plt.scatter(dataScraper[0],dataScraper[1], color='green')

**PCA: for dimension reduction**

In [18]:
X = np.asarray(vect_logs.todense())
pca = PCA()
pc = pca.fit_transform(X)
pc = pd.DataFrame(pc)
print(pc.shape)
pc

(1004, 15)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-82.400951,-65.475981,-10.698641,3.110510,0.074371,1.804762,-1.630317,1.276710,1.279009,-0.058238,-1.353269e-13,1.284833e-13,1.469421e-15,1.637615e-15,-3.460582e-17
1,-86.328597,-70.892918,-7.330865,-4.151772,0.512465,3.658031,1.677641,0.491920,0.131009,0.744773,1.394967e-12,3.354858e-14,1.006996e-15,2.666040e-17,8.970600e-18
2,-81.571565,-65.909247,-5.112635,3.811440,-4.595835,-1.728018,-0.974791,-1.237178,1.147646,0.551402,3.511759e-15,4.426455e-15,-4.311232e-15,3.644929e-15,6.946834e-16
3,-76.586237,-69.436151,-11.677718,-8.580253,0.840726,-0.505699,0.125948,3.307973,1.424642,0.788164,1.176729e-15,-6.426635e-14,1.013801e-14,5.018592e-15,-1.231352e-17
4,-84.485061,-75.598110,-9.322323,-0.554648,2.736410,3.323318,0.598066,0.939631,1.799176,-0.417409,8.920685e-15,-5.744123e-15,1.010097e-16,-3.669072e-16,1.566046e-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,-44.243627,-51.272235,14.606123,0.593727,6.729629,-6.114081,2.522183,2.207930,-0.996316,0.506522,-6.419342e-16,-9.156621e-16,7.681283e-17,-2.119685e-16,-1.146261e-17
1000,-11.624699,-17.963509,67.872671,-7.409804,4.740681,1.448092,-0.746958,-1.066630,-2.459040,-0.172239,9.754647e-16,1.514526e-15,3.828219e-16,4.241035e-17,5.322228e-18
1001,-86.569996,-76.560882,-8.869705,1.090989,1.019923,1.690733,1.737902,-0.647610,-0.583695,0.888286,-2.776408e-15,-1.502100e-15,-6.994105e-17,2.432109e-17,-1.411969e-18
1002,-86.868021,-70.965588,-12.575646,1.728894,-5.902138,0.924000,2.515823,-0.532754,-0.234317,0.419561,-6.027743e-15,3.548187e-17,1.673782e-16,-1.834555e-16,1.254380e-17


In [19]:
print("Eigen values: ", list(np.round(pca.explained_variance_, decimals=5)))
print("PCA_variance_ratio: ", pca.explained_variance_ratio_)
print(pca.n_components)

Eigen values:  [3841757.20995, 2408168.25817, 306.35298, 42.34812, 14.62254, 7.20666, 3.04214, 2.16623, 1.51048, 0.77505, 0.0, 0.0, 0.0, 0.0, 0.0]
PCA_variance_ratio:  [6.14651307e-01 3.85288212e-01 4.90140968e-05 6.77536995e-06
 2.33949334e-06 1.15301039e-06 4.86718381e-07 3.46580243e-07
 2.41664890e-07 1.24002567e-07 3.40745742e-34 6.37131685e-36
 4.69055192e-38 1.71815952e-38 1.00715205e-40]
None


Following code just serves as a way to experiment using sequence of numbers. Could we use tfidf for vectorizing data?

In [20]:
#Function to create a sequence of numbers. Approx. 10% of the sequences will consist of duplicates of the same number
import random

def create_sequence(arr, randVal):
    if (randVal < 1):
        val = random.randint(1,9)
        for i in range (50):
            arr.append(val)
    else:    
        for i in range (0, 50):
            arr.append(random.randint(1,9))
    return arr    

In [21]:
#Function to give a sequence a score depending on number of consecutive duplicates
def sequence_evaluation(df):
    arr = []
    for i in range (0, df.shape[0]):
        data = df.iloc[i].to_list()
        score = 0
        lr = 0.1
        val = data[0]
        for j in range (1, len(data)):
            if (data[j] == val):
                score -= lr*data[j]
                if(lr < 1):
                    lr += 1.1*lr
            else:
                score += data[j]
                val = data[j]
        arr.append(score) 
    return arr     

In [22]:
#Function that calculates length of the longest subsequence of consecutive duplicates for each sequence
from itertools import groupby

def longestConsecutive(df):
    arr = []
    for i in range (0, df.shape[0]):
        longest_streak = len(max([list(group) for _, group in groupby(df.iloc[i].to_list())], key=len))
        arr.append(longest_streak)
    return arr

In [23]:
#Creation of the sequence data frame
sequence_matrix = []
for i in range (1000):
    randVal = random.randint(0,9)
    arr = []
    sequence_matrix.append(create_sequence(arr, randVal))

sequence_df = pd.DataFrame(sequence_matrix)

sequence_df['score'] = sequence_evaluation(sequence_df)
sequence_df['longestConsec'] = longestConsecutive(sequence_df.loc[:, sequence_df.columns != 'score'])
sequence_df

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,score,longestConsec
0,7,1,8,7,3,1,8,9,8,7,...,9,6,6,8,2,6,6,4,217.58854,2
1,2,4,1,3,6,6,3,1,6,7,...,8,9,3,9,8,1,7,3,222.32210,2
2,7,6,8,2,9,2,4,6,4,2,...,6,4,8,6,1,5,3,8,185.05356,3
3,7,6,7,9,3,4,5,6,3,2,...,9,3,1,1,1,3,6,4,193.49558,3
4,9,2,7,5,4,4,1,2,8,9,...,3,3,8,5,6,6,9,7,212.83680,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,5,2,7,8,3,2,4,9,6,6,...,3,3,5,3,1,4,9,8,164.57694,3
996,5,2,3,9,3,9,9,4,7,4,...,9,4,1,2,6,9,5,6,187.94660,3
997,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,-267.58065,50
998,4,7,4,3,8,8,8,3,1,8,...,4,7,4,5,5,6,7,6,171.98876,4


In [24]:
#A yanky scatter plot....
plt.scatter(sequence_df['longestConsec'], sequence_df['score'])